# Mittelalterliche astronomische Diagramm Datenbank

Die Datenbank ist im  Datenformat JSON  in Repositorium der Edition Topoi publiziert.  Aus ihr wurde eine  flache Liste aller Diagramme  generiert, die über das Internet aus einem Git-Repositorium eingeladen wird.  Die Datenbank *diaattr.json* enthält als Forschungsdatenobjekt die Beschreibung der Diagramme mit Attributen, wie die Charakteristiken  der  Abbildungen und Manuskripte,  die Bibliothek des heutigen Halters  sowie die Stelle der Abbildung im Manuskript  sowie ihr Diagrammtyp.



In [65]:
import pandas as pd
import medDiaJson as medDia
import roman_date as roda

In [66]:
url = 'https://raw.githubusercontent.com/grasshoff/ComputationalHistoryScience/master/diaattr.json'
diagrams = pd.read_json(url)
diagrams.head(1)

,altID,author,biblio,diaAttr,diaID,diaTyp,diaURL,foliopage,manID,manURL,textID,textURL,year
0,Plin4,Plinius,"Vaticano BAV, Ross. 247","[{'M2.20': 0, 'M2.11': 0, 'M2.12': 0, 'M2.7': ...",MAPD0668,2,Dl_1&pn=396&dw=1858&dh=901&ww=0.8926&wh=0.6552...,200v,DL(1),Dl_1&pn=1,DL(1)_A,,IX


In [67]:
len(diagrams)

721

In [68]:
diagrams.keys()

Index(['altID', 'author', 'biblio', 'diaAttr', 'diaID', 'diaTyp', 'diaURL',
       'foliopage', 'manID', 'manURL', 'textID', 'textURL', 'year'],
      dtype='object')

 Insgesamt enthält die noch nicht weiter reduzierte ungefilterte Datenmenge die Beschreibung von {{len(diagrams)}}  Diagrammen.  (Bemerkung: die in diesem Satz angegebene Zahl wurde wie zuvor in der Python Zelle berechnet.)  Die Keys  mit dem Zusatz „ID“  sind eindeutige Kennzeichen für die benannten Entitäten, in diesem Fall  ein Diagramm, dass den Namen *diaID* oder alternativ *altID*  trägt, sowie der Text *textID* aus einem Manuskripts *manID*, der immer eindeutig aus dem Werk eines Autors stammt und mit dem Kennzeichen *textID*  benannt ist.
 
 \begin{tabular}{lc{12cm}}
 'altID'&  alternative ID  des Diagramms\\
 'author'&  antiker Autor des Werks\\
 'biblio'&  die  Bezeichnung des Werks der Bibliothek, in dem sich heute befindet\\
 'diaAttr'&  eine Liste mit Merkmalen des Diagramms, die manuell erstellt wurde\\
 'diaID'& ID des  Diagramms\\
 'diaTyp'&  Typ des Diagramms nach der Konvention des Projekts\\
 'diaURL'&   Teil einer URL, mit der das Diagramm aus dem Repositorium der Edition Topoi aufgerufen werden kann.\\
 'foliopage'&  Die Folioseite, auf der  sich das Diagramm  befindet\\
 'manID'& ID  des Manuskripts, in dem der Text mit den Diagramm enthalten ist\\
 'manURL'&  Teil einer URL für den Aufruf des Faksimile  des Manuskripts\\
 'textID'&ID  des Textes eines Autors\\
 'textURL'&  Teil einer URL für den Aufruf des Faksimile ist des Texts\\
 'year& geschätzte Zeit für die Abschrift \\
 \end{tabular}

# Dataframes

## Erweiterung der Felder: 'centuries'

 im ersten Schritt erweitern wir den Datensatz um eine neue Spalte 'century', für die das Jahrhundert aus der Angabe des Jahres berechnet wird. Die Datierung der Manuskripte wurde oft aufgrund der paleographischen Eigenheiten der Schrift in ein Jahrhundert datiert, oft ist es auch für ein bestimmtes Skriptorium oder gar Schreiber zu datieren gewesen und damit zeitlich enger einzugrenzen gewesen, in wenigen Ausnahmen konnte ein genaues Jahr für die Abfassung einer Abschrift gegeben werden. In der neuen Spalte werden diese Angaben vergleichbar auf das ganze Jahrhundert reduziert.

In [69]:
diagrams['century']=diagrams['year'].apply(lambda r: roda.from_roman(r))

In [70]:
diagrams.head(1)

,altID,author,biblio,diaAttr,diaID,diaTyp,diaURL,foliopage,manID,manURL,textID,textURL,year,century
0,Plin4,Plinius,"Vaticano BAV, Ross. 247","[{'M2.20': 0, 'M2.11': 0, 'M2.12': 0, 'M2.7': ...",MAPD0668,2,Dl_1&pn=396&dw=1858&dh=901&ww=0.8926&wh=0.6552...,200v,DL(1),Dl_1&pn=1,DL(1)_A,,IX,9


## Auswahl

Mit der Funktion reducedData(dataframe, keyValueList,debug=False)  lassen sich die Zeilen des Datensatzes auch solche reduzieren, deren Key  den String als Wert enthält. auf diese Weise kann schnell überprüft werden, ob bestimmte Sonderfälle oder Ausreißer der Daten vorkommen und im nächsten Schritt der Bereinigung des Datensatzes eliminiert werden sollen. Im ersten Fall wird überprüft, ob ein Jahrhundert entweder nicht angegeben ist und leer ist, oder ob der Wert mit dem Jahrhundert null angegeben ist.  In diesem Fall konnte  zur Datierung der Manuskripte keine zuverlässige Zeitangabe geschätzt werden.


In [71]:
medDia.reducedData(diagrams,[["century",0]])

,altID,author,biblio,diaAttr,diaID,diaTyp,diaURL,foliopage,manID,manURL,textID,textURL,year,century
0,Calc265,Calcidius,unbekannt,"[{'M10.7': 1, 'M10.15': 1, 'M10.33': 0, 'M10.2...",MAPD0186,10,Nw_1&pn=17&dw=1858&dh=901&ww=0.3051&wh=0.2919&...,15v,NW(1),Nw_1&pn=1,NW(1)_A,,unbekannt,0
1,Calc266,Calcidius,unbekannt,"[{'M11.50': 0, 'M11.53': 0, 'M11.52': 0, 'M11....",MAPD0187,11,Nw_1&pn=17&dw=1858&dh=901&ww=0.331&wh=0.3245&w...,16r,NW(1),Nw_1&pn=1,NW(1)_A,,unbekannt,0
2,Calc267,Calcidius,unbekannt,"[{'M12.3': 0, 'M12.35': 0, 'M12.2': 0, 'M12.37...",MAPD0188,12,Nw_1&pn=18&dw=1858&dh=901&ww=0.3514&wh=0.3788&...,16v,NW(1),Nw_1&pn=1,NW(1)_A,,unbekannt,0
3,Calc268,Calcidius,unbekannt,"[{'M13.5': 0, 'M13.1': 1, 'M13.8': 1, 'M13.16'...",MAPD0189,13,Nw_1&pn=19&dw=1858&dh=901&ww=0.2615&wh=0.2593&...,17v,NW(1),Nw_1&pn=1,NW(1)_A,,unbekannt,0
4,Calc269,Calcidius,unbekannt,"[{'M14.18': 0, 'M14.4': 1, 'M14.15': 0, 'M14.5...",MAPD0190,14,Nw_1&pn=23&dw=1858&dh=901&ww=0.2534&wh=0.2422&...,21v,NW(1),Nw_1&pn=1,NW(1)_A,,unbekannt,0
5,Calc270,Calcidius,unbekannt,"[{'M15.22': 0, 'M15.16': 1, 'M15.1': 1, 'M15.5...",MAPD0192,15,Nw_1&pn=23&dw=1858&dh=901&ww=0.282&wh=0.3136&w...,22r,NW(1),Nw_1&pn=1,NW(1)_A,,unbekannt,0


## Cleaning data
Leave out 
   - diagrams without corresponding diatyp
   - unclear year

In [72]:
diagrams=diagrams[diagrams["diaTyp"] != 0.0]
len(diagrams)

707

In [73]:
diagrams=diagrams[diagrams["diaTyp"] != '']
len(diagrams)

660

In [74]:
diagrams=diagrams[diagrams["century"] != 0]
anzahlDia=len(diagrams)
anzahlDia

654

In [75]:
diagrams.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 654 entries, 0 to 720
Data columns (total 14 columns):
altID        654 non-null object
author       654 non-null object
biblio       654 non-null object
diaAttr      654 non-null object
diaID        654 non-null object
diaTyp       654 non-null object
diaURL       654 non-null object
foliopage    654 non-null object
manID        654 non-null object
manURL       654 non-null object
textID       654 non-null object
textURL      654 non-null object
year         654 non-null object
century      654 non-null int64
dtypes: int64(1), object(13)
memory usage: 76.6+ KB


## Mengen: Autoren, years and centuries

In [76]:
setAutoren=set(pd.Series(diagrams["author"]))
setAutoren

{'Calcidius', 'Capella', 'Macrobius', 'Plinius'}

In [77]:
years=pd.Series(diagrams['year'])
setYears=set(years)
# setYears

In [78]:
setTextID=set(pd.Series(diagrams["textID"]))
len(setTextID)

229

In [79]:
setDiaTyp=set(pd.Series(diagrams["diaTyp"]))
setDiaTyp

{1.0,
 2.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 13.0,
 14.0,
 15.0,
 16.0,
 17.0,
 18.0,
 19.0,
 20.0,
 21.0,
 22.0,
 23.0,
 24.0,
 25.0,
 26.0,
 27.0,
 28.0,
 29.0,
 30.0,
 31.0}

In [80]:

setCent=set(pd.Series(diagrams["century"]))
setCent

{9, 10, 11, 12, 13, 14, 15}

## Häufigkeit und Verteilung der Diagramme

Die Datenbank enthält {{anzahlDia}}  Diagramme eines fest zugeordneten Typs. Insgesamt verteilen sich die Diagramme über die vier verschiedenen Autoren in folgender Häufigkeit. Die Anzahl der Diagramme ist für Calcidius fast doppelt so groß wie für die drei übrigen Autoren.

In [81]:
diagrams.groupby("author").size()

author
Calcidius    278
Capella      158
Macrobius    112
Plinius      106
dtype: int64

In [82]:
diagrams.groupby(["author","century"]).size()

author     century
Calcidius  9           18
           10          22
           11         106
           12          54
           13           7
           14           7
           15          64
Capella    9           69
           11          25
           12          30
           15          34
Macrobius  9            6
           10           5
           11          31
           12          54
           13           9
           14           2
           15           5
Plinius    9           30
           10          27
           11          22
           12          24
           13           2
           14           1
dtype: int64

In [83]:
pd.crosstab([diagrams["author"]],diagrams["century"],margins=True)

century,9,10,11,12,13,14,15,All
author,,,,,,,,
Calcidius,18,22,106,54,7,7,64,278
Capella,69,0,25,30,0,0,34,158
Macrobius,6,5,31,54,9,2,5,112
Plinius,30,27,22,24,2,1,0,106
All,123,54,184,162,18,10,103,654


In [84]:
pd.crosstab([diagrams["author"],diagrams["diaTyp"]],diagrams["century"],margins=True)

century             9  10   11   12  13  14   15  All
author    diaTyp                                     
Calcidius 10.0      3   4   17    9   1   1    9   44
          11.0      3   4   19   12   2   1   11   52
          12.0      3   3   18    9   0   1   10   44
          13.0      3   3   15    7   1   1    8   38
          14.0      3   4   20   10   2   2   13   54
          15.0      3   4   17    7   1   1   13   46
Capella   18.0      4   0    2    1   0   0    3   10
          19.0      6   0    2    0   0   0    0    8
          20.0      0   0    2    2   0   0    3    7
          21.0      8   0    0    2   0   0    0   10
          22.0      6   0    1    7   0   0    0   14
          23.0      5   0    2    1   0   0    3   11
          24.0      6   0    2    2   0   0    3   13
          25.0      5   0    2    1   0   0    3   11
          26.0      5   0    2    2   0   0    3   12
          27.0      5   0    2    2   0   0    3   12
          28.0      4   0    2    2   0   0    3   11
          29.0      5   0    2    3   0   0    3   13
          30.0      5   0    2    3   0   0    4   14
          31.0      5   0    2    2   0   0    3   12
Macrobius 8.0       6   5   14   17   1   1    1   45
          9.0       0   0    4   10   3   0    0   17
          16.0      0   0    5    9   3   0    1   18
          17.0      0   0    8   18   2   1    3   32
Plinius   1.0       3   6    8   10   1   0    0   28
          2.0       4   1    5    3   0   1    0   14
          3.0       6   7    6    3   1   0    0   23
          4.0       4   1    0    1   0   0    0    6
          5.0       2   7    1    2   0   0    0   12
          6.0       8   1    0    5   0   0    0   14
          7.0       3   4    2    0   0   0    0    9
All               123  54  184  162  18  10  103  654

## Häufigkeit und Verteilung der Texte mit Diagrammen

Die Datenbank enthält {{anzahlTexte}}  Diagramme eines fest zugeordneten Typs. Insgesamt verteilen sich die Diagramme über die vier verschiedenen Autoren in folgender Häufigkeit. Die Anzahl der Diagramme ist für Calcidius fast doppelt so groß wie für die drei übrigen Autoren.

In [85]:
texteVert=diagrams.groupby(["textID","century","author"]).size().to_frame(name='number dia').reset_index()
anzTexte=len(texteVert)

In [86]:
texteVert

,textID,century,author,number dia
0,AF(9)_A,9,Plinius,2
1,AH(9)_A,11,Calcidius,6
2,AH(9)_B,11,Plinius,2
3,AO(2)_A,11,Capella,12
4,AO(3)_A,12,Capella,11
5,AP(3)_A,9,Macrobius,1
6,AP(3)_B,9,Plinius,5
7,AP(7)_A,10,Plinius,1
8,AS(2)_A,9,Plinius,1
9,AX(2)_A,9,Plinius,1


In [87]:
texte=diagrams.groupby('textID').agg(lambda x: "|".join(x.sort_values().unique().tolist()))
texte.head(2)

,altID,author,biblio,diaID,diaURL,foliopage,manID,manURL,textURL,year
textID,,,,,,,,,,
AF(9)_A,Plin64|Plin78,Plinius,"Vaticano BAV, Vat. 645",MAPD0695|MAPD0709,Af_9&pn=63&dw=1858&dh=901&ww=0.454&wh=0.6583&w...,66v|67v,AF(9),Af_9&pn=1,,IX
AH(9)_A,Calc1|Calc2|Calc3|Calc4|Calc5|Calc6,Calcidius,"Vaticano BAV, Ms Regin. lat. 123",MAPD0001|MAPD0113|MAPD0210|MAPD0225|MAPD0236|M...,Ah_9&pn=59&dw=1858&dh=901&ww=0.1986&wh=0.2612&...,209r|210r|210v|211v|216v|217r,AH(9),Ah_9&pn=1,,XI(1056)


In [88]:
diagrams.groupby(["author","century","diaTyp"]).size()

author     century  diaTyp
Calcidius  9        10.0       3
                    11.0       3
                    12.0       3
                    13.0       3
                    14.0       3
                    15.0       3
           10       10.0       4
                    11.0       4
                    12.0       3
                    13.0       3
                    14.0       4
                    15.0       4
           11       10.0      17
                    11.0      19
                    12.0      18
                    13.0      15
                    14.0      20
                    15.0      17
           12       10.0       9
                    11.0      12
                    12.0       9
                    13.0       7
                    14.0      10
                    15.0       7
           13       10.0       1
                    11.0       2
                    13.0       1
                    14.0       2
                    15.0       1
           14   

## Counts

In [89]:
diagrams["author"].value_counts()

Calcidius    278
Capella      158
Macrobius    112
Plinius      106
Name: author, dtype: int64

In [90]:
diagrams["diaTyp"].value_counts()

14.0    54
11.0    52
15.0    46
8.0     45
10.0    44
12.0    44
13.0    38
17.0    32
1.0     28
3.0     23
16.0    18
9.0     17
2.0     14
30.0    14
22.0    14
6.0     14
29.0    13
24.0    13
27.0    12
5.0     12
31.0    12
26.0    12
28.0    11
25.0    11
23.0    11
21.0    10
18.0    10
7.0      9
19.0     8
20.0     7
4.0      6
Name: diaTyp, dtype: int64

In [91]:
diagrams["century"].value_counts()

11    184
12    162
9     123
15    103
10     54
13     18
14     10
Name: century, dtype: int64